In [13]:
import os
import pycolmap
import numpy as np
import open3d as o3d

# Set the path to the dataset
# dataset_path = "/Users/kp/Workspace/Git_Repos/Structure_From_Motion/Datasets/fountain"
dataset_path = ".././Datasets/fountain"
# Create a COLMAP database
database_path = os.path.join(dataset_path, "database.db")
db = pycolmap.Database(database_path)

# Configure SIFT feature extraction options
sift_options = pycolmap.SiftExtractionOptions()
sift_options.max_num_features = 8192
sift_options.first_octave = -1
sift_options.num_octaves = 4

# Configure image reader options
reader_options = pycolmap.ImageReaderOptions()

# Extract features
pycolmap.extract_features(database_path, dataset_path, image_list=[],
                          camera_mode=pycolmap.CameraMode.AUTO,
                          camera_model="OPENCV",
                          reader_options=reader_options,
                          sift_options=sift_options)

# Match features
pycolmap.match_exhaustive(database_path)

# Configure incremental mapping options
pipeline_options = pycolmap.IncrementalPipelineOptions()
pipeline_options.mapper.num_threads = 4
pipeline_options.mapper.init_min_num_inliers = 100
pipeline_options.mapper.abs_pose_min_num_inliers = 30
pipeline_options.mapper.abs_pose_min_inlier_ratio = 0.25

# Set up output path
output_path = os.path.join(dataset_path, "reconstruction")
os.makedirs(output_path, exist_ok=True)

# Run incremental mapping
reconstructions = pycolmap.incremental_mapping(database_path, dataset_path, output_path, options=pipeline_options)

if reconstructions:
    reconstruction = next(iter(reconstructions.values()))
    print(f"Reconstruction completed and saved to {output_path}")
    print(f"Number of registered images: {reconstruction.num_reg_images()}")
    print(f"Number of 3D points: {reconstruction.num_points3D()}")

    # Save reconstruction as text files
    reconstruction.write_text(output_path)

    # Prepare point cloud data
    points = []
    colors = []

    for point3D_id in reconstruction.points3D:
        point3D = reconstruction.points3D[point3D_id]
        points.append(point3D.xyz)
        colors.append(point3D.color)

    points = np.array(points)
    colors = np.array(colors)

    # Create Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)

    # Post-processing: downsampling and outlier removal
    pcd = pcd.voxel_down_sample(voxel_size=0.01)
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

    # Save point cloud as PLY file
    ply_path = os.path.join(output_path, "reconstruction.ply")
    o3d.io.write_point_cloud(ply_path, pcd)
    print(f"Point cloud saved as PLY file: {ply_path}")

    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd])

else:
    print("No reconstruction was created.")

W20240818 21:35:47.366699 0x1f5cd0f40 feature_extraction.cc:406] Your current options use the maximum number of threads on the machine to extract features. Extracting SIFT features on the CPU can consume a lot of RAM per thread for large images. Consider reducing the maximum image size and/or the first octave or manually limit the number of extraction threads. Ignore this warning, if your machine has sufficient memory for the current settings.
I20240818 21:35:47.366809 0x3108eb000 misc.cc:198] 
Feature extraction
I20240818 21:35:47.366995 0x3636fb000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:35:47.367008 0x364627000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:35:47.367015 0x3646b3000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:35:47.367029 0x364913000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:35:47.367037 0x364887000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:35:47.367038 0x36499f000 sift.cc:

Reconstruction completed and saved to .././Datasets/fountain/reconstruction
Number of registered images: 11
Number of 3D points: 14225
Point cloud saved as PLY file: .././Datasets/fountain/reconstruction/reconstruction.ply
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [14]:
import os
import pycolmap
import numpy as np
import open3d as o3d
import subprocess

# ... existing code ...

if reconstructions:
    reconstruction = next(iter(reconstructions.values()))
    print(f"Reconstruction completed and saved to {output_path}")
    print(f"Number of registered images: {reconstruction.num_reg_images()}")
    print(f"Number of 3D points: {reconstruction.num_points3D()}")

    # Save reconstruction as text files
    reconstruction.write_text(output_path)

    # Run dense reconstruction using COLMAP command-line
    colmap_bin = "colmap"  # Adjust this path if necessary

    # Undistort images
    subprocess.run([colmap_bin, "image_undistorter", 
                    "--image_path", dataset_path,
                    "--input_path", output_path,
                    "--output_path", os.path.join(output_path, "dense"),
                    "--output_type", "COLMAP"])

    # Run dense stereo
    subprocess.run([colmap_bin, "patch_match_stereo", 
                    "--workspace_path", os.path.join(output_path, "dense")])

    # Run stereo fusion
    subprocess.run([colmap_bin, "stereo_fusion", 
                    "--workspace_path", os.path.join(output_path, "dense"),
                    "--input_type", "geometric",
                    "--output_path", os.path.join(output_path, "dense", "fused.ply")])

    # Run Poisson surface reconstruction
    subprocess.run([colmap_bin, "poisson_mesher", 
                    "--input_path", os.path.join(output_path, "dense", "fused.ply"),
                    "--output_path", os.path.join(output_path, "dense", "meshed-poisson.ply")])

    # Run texture mapping
    subprocess.run([colmap_bin, "model_converter", 
                    "--input_path", output_path,
                    "--output_path", os.path.join(output_path, "dense", "sparse"),
                    "--output_type", "TXT"])

    subprocess.run([colmap_bin, "image_undistorter", 
                    "--image_path", dataset_path,
                    "--input_path", os.path.join(output_path, "dense", "sparse"),
                    "--output_path", os.path.join(output_path, "dense", "images"),
                    "--output_type", "COLMAP"])

    subprocess.run([colmap_bin, "texture_mapping", 
                    "--workspace_path", os.path.join(output_path, "dense", "images"),
                    "--input_path", os.path.join(output_path, "dense", "meshed-poisson.ply"),
                    "--output_path", os.path.join(output_path, "dense", "textured.obj")])

    # Load the textured mesh
    textured_mesh_path = os.path.join(output_path, "dense", "textured.obj")
    textured_mesh = o3d.io.read_triangle_mesh(textured_mesh_path)

    # Ensure the mesh has vertex colors (required for visualization)
    if not textured_mesh.has_vertex_colors():
        textured_mesh.vertex_colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(len(textured_mesh.vertices), 3)))

    # Visualize the textured mesh
    o3d.visualization.draw_geometries([textured_mesh])

    print(f"Textured mesh saved as: {textured_mesh_path}")

else:
    print("No reconstruction was created.")

Reconstruction completed and saved to .././Datasets/fountain/reconstruction
Number of registered images: 11
Number of 3D points: 14225


FileNotFoundError: [Errno 2] No such file or directory: 'colmap'

In [ ]:
import os
import pycolmap
import numpy as np
import open3d as o3d
import subprocess

# Set the path to the dataset
dataset_path = "Datasets/fountain"

# Create a COLMAP database
database_path = os.path.join(dataset_path, "database.db")
db = pycolmap.Database(database_path)

# Configure SIFT feature extraction options
sift_options = pycolmap.SiftExtractionOptions()
sift_options.max_num_features = 8192
sift_options.first_octave = -1
sift_options.num_octaves = 4

# Configure image reader options
reader_options = pycolmap.ImageReaderOptions()

# Extract features
pycolmap.extract_features(database_path, dataset_path, image_list=[],
                          camera_mode=pycolmap.CameraMode.AUTO,
                          camera_model="OPENCV",
                          reader_options=reader_options,
                          sift_options=sift_options)

# Match features
pycolmap.match_exhaustive(database_path)

# Configure incremental mapping options
pipeline_options = pycolmap.IncrementalPipelineOptions()
pipeline_options.mapper.num_threads = 4
pipeline_options.mapper.init_min_num_inliers = 100
pipeline_options.mapper.abs_pose_min_num_inliers = 30
pipeline_options.mapper.abs_pose_min_inlier_ratio = 0.25

# Set up output path
output_path = os.path.join(dataset_path, "reconstruction")
os.makedirs(output_path, exist_ok=True)

# Run incremental mapping
reconstructions = pycolmap.incremental_mapping(database_path, dataset_path, output_path, options=pipeline_options)

if reconstructions:
    reconstruction = next(iter(reconstructions.values()))
    print(f"Reconstruction completed and saved to {output_path}")
    print(f"Number of registered images: {reconstruction.num_reg_images()}")
    print(f"Number of 3D points: {reconstruction.num_points3D()}")

    # Save reconstruction as text files
    reconstruction.write_text(output_path)

    # Run dense reconstruction using COLMAP command-line
    colmap_bin = "colmap"  # Adjust this path if necessary

    # Undistort images
    subprocess.run([colmap_bin, "image_undistorter", 
                    "--image_path", dataset_path,
                    "--input_path", output_path,
                    "--output_path", os.path.join(output_path, "dense"),
                    "--output_type", "COLMAP"])

    # Run dense stereo
    subprocess.run([colmap_bin, "patch_match_stereo", 
                    "--workspace_path", os.path.join(output_path, "dense")])

    # Run stereo fusion
    subprocess.run([colmap_bin, "stereo_fusion", 
                    "--workspace_path", os.path.join(output_path, "dense"),
                    "--input_type", "geometric",
                    "--output_path", os.path.join(output_path, "dense", "fused.ply")])

    # Run Poisson surface reconstruction
    subprocess.run([colmap_bin, "poisson_mesher", 
                    "--input_path", os.path.join(output_path, "dense", "fused.ply"),
                    "--output_path", os.path.join(output_path, "dense", "meshed-poisson.ply")])

    # Run texture mapping
    subprocess.run([colmap_bin, "model_converter", 
                    "--input_path", output_path,
                    "--output_path", os.path.join(output_path, "dense", "sparse"),
                    "--output_type", "TXT"])

    subprocess.run([colmap_bin, "image_undistorter", 
                    "--image_path", dataset_path,
                    "--input_path", os.path.join(output_path, "dense", "sparse"),
                    "--output_path", os.path.join(output_path, "dense", "images"),
                    "--output_type", "COLMAP"])

    subprocess.run([colmap_bin, "texture_mapping", 
                    "--workspace_path", os.path.join(output_path, "dense", "images"),
                    "--input_path", os.path.join(output_path, "dense", "meshed-poisson.ply"),
                    "--output_path", os.path.join(output_path, "dense", "textured.obj")])

    # Load the textured mesh
    textured_mesh_path = os.path.join(output_path, "dense", "textured.obj")
    textured_mesh = o3d.io.read_triangle_mesh(textured_mesh_path)

    # Ensure the mesh has vertex colors (required for visualization)
    if not textured_mesh.has_vertex_colors():
        textured_mesh.vertex_colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(len(textured_mesh.vertices), 3)))

    # Visualize the textured mesh
    o3d.visualization.draw_geometries([textured_mesh])

    print(f"Textured mesh saved as: {textured_mesh_path}")

else:
    print("No reconstruction was created.")

E20240818 21:28:10.211627 0x1f5cd0f40 database.cc:263] SQLite error: unable to open database file


RuntimeError: [database.cc:263] SQLite error: unable to open database file

In [ ]:
import os
import pycolmap
import numpy as np
import open3d as o3d
import subprocess

# Set the path to the dataset
dataset_path = "Datasets/fountain"

# Create a COLMAP database
database_path = os.path.join(dataset_path, "database.db")
db = pycolmap.Database(database_path)

# Configure SIFT feature extraction options
sift_options = pycolmap.SiftExtractionOptions()
sift_options.max_num_features = 8192
sift_options.first_octave = -1
sift_options.num_octaves = 4

# Configure image reader options
reader_options = pycolmap.ImageReaderOptions()

# Extract features
pycolmap.extract_features(database_path, dataset_path, image_list=[],
                          camera_mode=pycolmap.CameraMode.AUTO,
                          camera_model="OPENCV",
                          reader_options=reader_options,
                          sift_options=sift_options)

# Match features
pycolmap.match_exhaustive(database_path)

# Configure incremental mapping options
pipeline_options = pycolmap.IncrementalPipelineOptions()
pipeline_options.mapper.num_threads = 4
pipeline_options.mapper.init_min_num_inliers = 100
pipeline_options.mapper.abs_pose_min_num_inliers = 30
pipeline_options.mapper.abs_pose_min_inlier_ratio = 0.25

# Set up output path
output_path = os.path.join(dataset_path, "reconstruction")
os.makedirs(output_path, exist_ok=True)

# Run incremental mapping
reconstructions = pycolmap.incremental_mapping(database_path, dataset_path, output_path, options=pipeline_options)

if reconstructions:
    reconstruction = next(iter(reconstructions.values()))
    print(f"Reconstruction completed and saved to {output_path}")
    print(f"Number of registered images: {reconstruction.num_reg_images()}")
    print(f"Number of 3D points: {reconstruction.num_points3D()}")

    # Save reconstruction as text files
    reconstruction.write_text(output_path)

    # Run dense reconstruction using COLMAP command-line
    colmap_bin = "colmap"  # Adjust this path if necessary

    # Undistort images
    subprocess.run([colmap_bin, "image_undistorter", 
                    "--image_path", dataset_path,
                    "--input_path", output_path,
                    "--output_path", os.path.join(output_path, "dense"),
                    "--output_type", "COLMAP"])

    # Run dense stereo
    subprocess.run([colmap_bin, "patch_match_stereo", 
                    "--workspace_path", os.path.join(output_path, "dense")])

    # Run stereo fusion
    subprocess.run([colmap_bin, "stereo_fusion", 
                    "--workspace_path", os.path.join(output_path, "dense"),
                    "--input_type", "geometric",
                    "--output_path", os.path.join(output_path, "dense", "fused.ply")])

    # Run Poisson surface reconstruction
    subprocess.run([colmap_bin, "poisson_mesher", 
                    "--input_path", os.path.join(output_path, "dense", "fused.ply"),
                    "--output_path", os.path.join(output_path, "dense", "meshed-poisson.ply")])

    # Run texture mapping
    subprocess.run([colmap_bin, "model_converter", 
                    "--input_path", output_path,
                    "--output_path", os.path.join(output_path, "dense", "sparse"),
                    "--output_type", "TXT"])

    subprocess.run([colmap_bin, "image_undistorter", 
                    "--image_path", dataset_path,
                    "--input_path", os.path.join(output_path, "dense", "sparse"),
                    "--output_path", os.path.join(output_path, "dense", "images"),
                    "--output_type", "COLMAP"])

    subprocess.run([colmap_bin, "texture_mapping", 
                    "--workspace_path", os.path.join(output_path, "dense", "images"),
                    "--input_path", os.path.join(output_path, "dense", "meshed-poisson.ply"),
                    "--output_path", os.path.join(output_path, "dense", "textured.obj")])

    # Load the textured mesh
    textured_mesh_path = os.path.join(output_path, "dense", "textured.obj")
    textured_mesh = o3d.io.read_triangle_mesh(textured_mesh_path)

    # Ensure the mesh has vertex colors (required for visualization)
    if not textured_mesh.has_vertex_colors():
        textured_mesh.vertex_colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(len(textured_mesh.vertices), 3)))

    # Visualize the textured mesh
    o3d.visualization.draw_geometries([textured_mesh])

    print(f"Textured mesh saved as: {textured_mesh_path}")

else:
    print("No reconstruction was created.")

E20240818 21:28:21.804961 0x1f5cd0f40 database.cc:263] SQLite error: unable to open database file


RuntimeError: [database.cc:263] SQLite error: unable to open database file

In [ ]:
import os
import pycolmap
import numpy as np
import open3d as o3d
import subprocess

# Set the path to the dataset
dataset_path = "Datasets/fountain"

# Create a COLMAP database
database_path = os.path.join(dataset_path, "database.db")
db = pycolmap.Database(database_path)

# Configure SIFT feature extraction options
sift_options = pycolmap.SiftExtractionOptions()
sift_options.max_num_features = 8192
sift_options.first_octave = -1
sift_options.num_octaves = 4

# Configure image reader options
reader_options = pycolmap.ImageReaderOptions()

# Extract features
pycolmap.extract_features(database_path, dataset_path, image_list=[],
                          camera_mode=pycolmap.CameraMode.AUTO,
                          camera_model="OPENCV",
                          reader_options=reader_options,
                          sift_options=sift_options)

# Match features
pycolmap.match_exhaustive(database_path)

# Configure incremental mapping options
pipeline_options = pycolmap.IncrementalPipelineOptions()
pipeline_options.mapper.num_threads = 4
pipeline_options.mapper.init_min_num_inliers = 100
pipeline_options.mapper.abs_pose_min_num_inliers = 30
pipeline_options.mapper.abs_pose_min_inlier_ratio = 0.25

# Set up output path
output_path = os.path.join(dataset_path, "reconstruction")
os.makedirs(output_path, exist_ok=True)

# Run incremental mapping
reconstructions = pycolmap.incremental_mapping(database_path, dataset_path, output_path, options=pipeline_options)

if reconstructions:
    reconstruction = next(iter(reconstructions.values()))
    print(f"Reconstruction completed and saved to {output_path}")
    print(f"Number of registered images: {reconstruction.num_reg_images()}")
    print(f"Number of 3D points: {reconstruction.num_points3D()}")

    # Save reconstruction as text files
    reconstruction.write_text(output_path)

    # Run dense reconstruction using COLMAP command-line
    colmap_bin = "colmap"  # Adjust this path if necessary

    # Undistort images
    subprocess.run([colmap_bin, "image_undistorter", 
                    "--image_path", dataset_path,
                    "--input_path", output_path,
                    "--output_path", os.path.join(output_path, "dense"),
                    "--output_type", "COLMAP"])

    # Run dense stereo
    subprocess.run([colmap_bin, "patch_match_stereo", 
                    "--workspace_path", os.path.join(output_path, "dense")])

    # Run stereo fusion
    subprocess.run([colmap_bin, "stereo_fusion", 
                    "--workspace_path", os.path.join(output_path, "dense"),
                    "--input_type", "geometric",
                    "--output_path", os.path.join(output_path, "dense", "fused.ply")])

    # Run Poisson surface reconstruction
    subprocess.run([colmap_bin, "poisson_mesher", 
                    "--input_path", os.path.join(output_path, "dense", "fused.ply"),
                    "--output_path", os.path.join(output_path, "dense", "meshed-poisson.ply")])

    # Run texture mapping
    subprocess.run([colmap_bin, "model_converter", 
                    "--input_path", output_path,
                    "--output_path", os.path.join(output_path, "dense", "sparse"),
                    "--output_type", "TXT"])

    subprocess.run([colmap_bin, "image_undistorter", 
                    "--image_path", dataset_path,
                    "--input_path", os.path.join(output_path, "dense", "sparse"),
                    "--output_path", os.path.join(output_path, "dense", "images"),
                    "--output_type", "COLMAP"])

    subprocess.run([colmap_bin, "texture_mapping", 
                    "--workspace_path", os.path.join(output_path, "dense", "images"),
                    "--input_path", os.path.join(output_path, "dense", "meshed-poisson.ply"),
                    "--output_path", os.path.join(output_path, "dense", "textured.obj")])

    # Load the textured mesh
    textured_mesh_path = os.path.join(output_path, "dense", "textured.obj")
    textured_mesh = o3d.io.read_triangle_mesh(textured_mesh_path)

    # Ensure the mesh has vertex colors (required for visualization)
    if not textured_mesh.has_vertex_colors():
        textured_mesh.vertex_colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(len(textured_mesh.vertices), 3)))

    # Visualize the textured mesh
    o3d.visualization.draw_geometries([textured_mesh])

    print(f"Textured mesh saved as: {textured_mesh_path}")

else:
    print("No reconstruction was created.")

W20240818 00:06:57.918786 0x1f5cd0f40 feature_extraction.cc:406] Your current options use the maximum number of threads on the machine to extract features. Extracting SIFT features on the CPU can consume a lot of RAM per thread for large images. Consider reducing the maximum image size and/or the first octave or manually limit the number of extraction threads. Ignore this warning, if your machine has sufficient memory for the current settings.
I20240818 00:06:57.918880 0x33e6bb000 misc.cc:198] 
Feature extraction
I20240818 00:06:57.919052 0x365fa3000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 00:06:57.919071 0x3660bb000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 00:06:57.919069 0x36602f000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 00:06:57.919095 0x3661d3000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 00:06:57.919099 0x36625f000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 00:06:57.919109 0x3662eb000 sift.cc:

Reconstruction completed and saved to Datasets/fountain/reconstruction
Number of registered images: 11
Number of 3D points: 14225


FileNotFoundError: [Errno 2] No such file or directory: 'colmap'

In [ ]:
import os
import pycolmap
import numpy as np
import open3d as o3d

# Set the path to the dataset
dataset_path = ".././Datasets/fountain"

# Create a COLMAP database
database_path = os.path.join(dataset_path, "database.db")
db = pycolmap.Database(database_path)

# Configure SIFT feature extraction options
sift_options = pycolmap.SiftExtractionOptions()
sift_options.max_num_features = 8192
sift_options.first_octave = -1
sift_options.num_octaves = 4

# Configure image reader options
reader_options = pycolmap.ImageReaderOptions()

# Extract features
pycolmap.extract_features(database_path, dataset_path, image_list=[],
                          camera_mode=pycolmap.CameraMode.AUTO,
                          camera_model="OPENCV",
                          reader_options=reader_options,
                          sift_options=sift_options)

# Match features
pycolmap.match_exhaustive(database_path)

# Configure incremental mapping options
pipeline_options = pycolmap.IncrementalPipelineOptions()
pipeline_options.mapper.num_threads = 4
pipeline_options.mapper.init_min_num_inliers = 100
pipeline_options.mapper.abs_pose_min_num_inliers = 30
pipeline_options.mapper.abs_pose_min_inlier_ratio = 0.25

# Set up output path
output_path = os.path.join(dataset_path, "reconstruction")
os.makedirs(output_path, exist_ok=True)

# Run incremental mapping
reconstructions = pycolmap.incremental_mapping(database_path, dataset_path, output_path, options=pipeline_options)

if reconstructions:
    reconstruction = next(iter(reconstructions.values()))
    print(f"Reconstruction completed and saved to {output_path}")
    print(f"Number of registered images: {reconstruction.num_reg_images()}")
    print(f"Number of 3D points: {reconstruction.num_points3D()}")

    # Save reconstruction as text files
    reconstruction.write_text(output_path)

    # Prepare point cloud data
    points = []
    colors = []

    for point3D_id in reconstruction.points3D:
        point3D = reconstruction.points3D[point3D_id]
        points.append(point3D.xyz)
        colors.append(point3D.color)

    points = np.array(points)
    colors = np.array(colors)

    # Create Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)

    # Post-processing: downsampling and outlier removal
    pcd = pcd.voxel_down_sample(voxel_size=0.01)
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

    # Create mesh from point cloud
    mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)

    # Simplify mesh
    mesh = mesh.simplify_quadric_decimation(target_number_of_triangles=100000)

    # Estimate vertex normals for better visualization
    mesh.compute_vertex_normals()

    # Save mesh as OBJ file
    mesh_path = os.path.join(output_path, "reconstructed_mesh.obj")
    o3d.io.write_triangle_mesh(mesh_path, mesh)
    print(f"Mesh saved as: {mesh_path}")

    # Visualize the mesh
    o3d.visualization.draw_geometries([mesh])

else:
    print("No reconstruction was created.")

W20240818 21:29:47.603452 0x1f5cd0f40 feature_extraction.cc:406] Your current options use the maximum number of threads on the machine to extract features. Extracting SIFT features on the CPU can consume a lot of RAM per thread for large images. Consider reducing the maximum image size and/or the first octave or manually limit the number of extraction threads. Ignore this warning, if your machine has sufficient memory for the current settings.
I20240818 21:29:47.603711 0x3108eb000 misc.cc:198] 
Feature extraction
I20240818 21:29:47.603932 0x332c43000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:29:47.603940 0x332ccf000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:29:47.604084 0x332d5b000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:29:47.604206 0x334de7000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:29:47.604251 0x33ee73000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:29:47.604456 0x344187000 sift.cc:

Reconstruction completed and saved to .././Datasets/fountain/reconstruction
Number of registered images: 11
Number of 3D points: 14225


RuntimeError: [1;31m[Open3D Error] (static std::tuple<std::shared_ptr<TriangleMesh>, std::vector<double>> open3d::geometry::TriangleMesh::CreateFromPointCloudPoisson(const PointCloud &, size_t, float, float, bool, int)) /Users/renes/development/open3d_work/Open3D/cpp/open3d/geometry/SurfaceReconstructionPoisson.cpp:732: Point cloud has no normals
[0;m

In [ ]:
import os
import pycolmap
import numpy as np
import open3d as o3d

# Set the path to the dataset
dataset_path = ".././Datasets/fountain"

# Create a COLMAP database
database_path = os.path.join(dataset_path, "database.db")
db = pycolmap.Database(database_path)

# Configure SIFT feature extraction options
sift_options = pycolmap.SiftExtractionOptions()
sift_options.max_num_features = 8192
sift_options.first_octave = -1
sift_options.num_octaves = 4

# Configure image reader options
reader_options = pycolmap.ImageReaderOptions()

# Extract features
pycolmap.extract_features(database_path, dataset_path, image_list=[],
                          camera_mode=pycolmap.CameraMode.AUTO,
                          camera_model="OPENCV",
                          reader_options=reader_options,
                          sift_options=sift_options)

# Match features
pycolmap.match_exhaustive(database_path)

# Configure incremental mapping options
pipeline_options = pycolmap.IncrementalPipelineOptions()
pipeline_options.mapper.num_threads = 4
pipeline_options.mapper.init_min_num_inliers = 100
pipeline_options.mapper.abs_pose_min_num_inliers = 30
pipeline_options.mapper.abs_pose_min_inlier_ratio = 0.25

# Set up output path
output_path = os.path.join(dataset_path, "reconstruction")
os.makedirs(output_path, exist_ok=True)

# Run incremental mapping
reconstructions = pycolmap.incremental_mapping(database_path, dataset_path, output_path, options=pipeline_options)

if reconstructions:
    reconstruction = next(iter(reconstructions.values()))
    print(f"Reconstruction completed and saved to {output_path}")
    print(f"Number of registered images: {reconstruction.num_reg_images()}")
    print(f"Number of 3D points: {reconstruction.num_points3D()}")

    # Save reconstruction as text files
    reconstruction.write_text(output_path)

    # Prepare point cloud data
    points = []
    colors = []

    for point3D_id in reconstruction.points3D:
        point3D = reconstruction.points3D[point3D_id]
        points.append(point3D.xyz)
        colors.append(point3D.color)

    points = np.array(points)
    colors = np.array(colors)

    # Create Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)

    # Post-processing: downsampling and outlier removal
    pcd = pcd.voxel_down_sample(voxel_size=0.01)
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

    # Estimate normals
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    pcd.orient_normals_consistent_tangent_plane(100)

    # Visualize the point cloud (optional, for debugging)
    # o3d.visualization.draw_geometries([pcd])

    # Create mesh from point cloud
    print("Creating mesh from point cloud...")
    mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)

    # Remove low-density vertices
    print("Removing low-density vertices...")
    vertices_to_remove = mesh.remove_vertices_by_mask(mesh.vertex_density() < 0.01)
    print(f"Removed {vertices_to_remove} low-density vertices")

    # Simplify mesh
    print("Simplifying mesh...")
    mesh = mesh.simplify_quadric_decimation(target_number_of_triangles=100000)

    # Estimate vertex normals for better visualization
    mesh.compute_vertex_normals()

    # Save mesh as OBJ file
    mesh_path = os.path.join(output_path, "reconstructed_mesh.obj")
    o3d.io.write_triangle_mesh(mesh_path, mesh)
    print(f"Mesh saved as: {mesh_path}")

    # Visualize the mesh
    print("Visualizing the mesh...")
    o3d.visualization.draw_geometries([mesh])

else:
    print("No reconstruction was created.")

W20240818 21:34:44.066395 0x1f5cd0f40 feature_extraction.cc:406] Your current options use the maximum number of threads on the machine to extract features. Extracting SIFT features on the CPU can consume a lot of RAM per thread for large images. Consider reducing the maximum image size and/or the first octave or manually limit the number of extraction threads. Ignore this warning, if your machine has sufficient memory for the current settings.
I20240818 21:34:44.066513 0x3108eb000 misc.cc:198] 
Feature extraction
I20240818 21:34:44.066778 0x3625c3000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:34:44.066778 0x3636fb000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:34:44.066872 0x364b87000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:34:44.066878 0x364c13000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:34:44.066857 0x36499f000 sift.cc:722] Creating SIFT CPU feature extractor
I20240818 21:34:44.066897 0x364d2b000 sift.cc:

Reconstruction completed and saved to .././Datasets/fountain/reconstruction
Number of registered images: 11
Number of 3D points: 14225
Creating mesh from point cloud...
Removing low-density vertices...


AttributeError: 'open3d.cpu.pybind.geometry.TriangleMesh' object has no attribute 'vertex_density'

In [11]:
import os
import pycolmap
import numpy as np
import open3d as o3d

# Set the path to the dataset
dataset_path = "Datasets/fountain"

# Create a COLMAP database
database_path = os.path.join(dataset_path, "database.db")
db = pycolmap.Database(database_path)

# Configure SIFT feature extraction options
sift_options = pycolmap.SiftExtractionOptions()
sift_options.max_num_features = 8192
sift_options.first_octave = -1
sift_options.num_octaves = 4

# Configure image reader options
reader_options = pycolmap.ImageReaderOptions()

# Extract features
pycolmap.extract_features(database_path, dataset_path, image_list=[],
                          camera_mode=pycolmap.CameraMode.AUTO,
                          camera_model="OPENCV",
                          reader_options=reader_options,
                          sift_options=sift_options)

# Match features
pycolmap.match_exhaustive(database_path)

# Configure incremental mapping options
pipeline_options = pycolmap.IncrementalPipelineOptions()
pipeline_options.mapper.num_threads = 4
pipeline_options.mapper.init_min_num_inliers = 100
pipeline_options.mapper.abs_pose_min_num_inliers = 30
pipeline_options.mapper.abs_pose_min_inlier_ratio = 0.25

# Set up output path
output_path = os.path.join(dataset_path, "reconstruction")
os.makedirs(output_path, exist_ok=True)

# Run incremental mapping
reconstructions = pycolmap.incremental_mapping(database_path, dataset_path, output_path, options=pipeline_options)

if reconstructions:
    reconstruction = next(iter(reconstructions.values()))
    print(f"Reconstruction completed and saved to {output_path}")
    print(f"Number of registered images: {reconstruction.num_reg_images()}")
    print(f"Number of 3D points: {reconstruction.num_points3D()}")

    # Save reconstruction as text files
    reconstruction.write_text(output_path)

    # Prepare point cloud data
    points = []
    colors = []

    for point3D_id in reconstruction.points3D:
        point3D = reconstruction.points3D[point3D_id]
        points.append(point3D.xyz)
        colors.append(point3D.color)

    points = np.array(points)
    colors = np.array(colors)

    # Create Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)

    # Post-processing: downsampling and outlier removal
    pcd = pcd.voxel_down_sample(voxel_size=0.01)
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

    # Estimate normals
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    pcd.orient_normals_consistent_tangent_plane(100)

    # Visualize the point cloud (optional, for debugging)
    # o3d.visualization.draw_geometries([pcd])

    # Create mesh from point cloud
    print("Creating mesh from point cloud...")
    mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)

    # Simplify mesh
    print("Simplifying mesh...")
    mesh = mesh.simplify_quadric_decimation(target_number_of_triangles=100000)

    # Estimate vertex normals for better visualization
    mesh.compute_vertex_normals()

    # Save mesh as OBJ file
    mesh_path = os.path.join(output_path, "reconstructed_mesh.obj")
    o3d.io.write_triangle_mesh(mesh_path, mesh)
    print(f"Mesh saved as: {mesh_path}")

    # Visualize the mesh
    print("Visualizing the mesh...")
    o3d.visualization.draw_geometries([mesh])

else:
    print("No reconstruction was created.")

E20240818 21:34:22.533201 0x1f5cd0f40 database.cc:263] SQLite error: unable to open database file


RuntimeError: [database.cc:263] SQLite error: unable to open database file